# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





In [5]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re
from selenium.webdriver.support.ui import Select
from datetime import datetime

In [ ]:
#extension
options = webdriver.ChromeOptions()
options.add_extension('/Users/mengchen/ChroPath.crx')

#open
url = 'http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx'
browser = webdriver.Chrome(executable_path='/Users/mengchen/chromedriver',options = options)
browser.get(url)
browser.maximize_window()

In [7]:
#select&click_btn

selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
ser_btn = browser.find_element_by_id('ctl05_btnQuery')


selectYear.select_by_value('2018')
selectSite.select_by_value('11')
ser_btn.click()

In [36]:
#find main table
soup = BeautifulSoup(browser.page_source, 'html.parser')
table = soup.find('table', class_='TABLE_G')

#parse
df = pd.read_html(str(table))[0]
df = df.loc[:,['監測日期','監測項目','監測值']]
df.dropna(inplace=True)
df = df.pivot(index='監測日期',columns='監測項目')
df.columns = df.columns.get_level_values(1)
df.index = pd.DatetimeIndex(df.index).strftime('%Y/%m')
df.index.rename('DATE',inplace=True)

df

監測項目,CO,NO,NO2,NOx,O3,PM10,SO2
DATE,,,,,,,
2018/01,0.34,3.41,10.77,14.17,33.4,23.0,1.8
2018/02,0.44,4.48,14.95,19.43,32.5,41.0,1.9
2018/03,0.40,4.93,13.56,18.50,35.0,39.0,2.2
2018/04,0.38,2.60,12.64,15.24,38.4,48.0,2.3
2018/05,0.34,2.26,12.94,15.20,31.6,37.0,3.1
2018/06,0.29,2.41,11.38,13.79,29.5,26.0,2.7
2018/07,0.21,2.68,8.92,11.60,18.7,24.0,2.2
2018/08,0.30,2.68,12.08,14.75,26.4,26.0,2.4
2018/09,0.26,2.41,10.00,12.40,29.1,28.0,2.1


### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [38]:
df.loc['2018/01':'2018/08','SO2']

DATE
2018/01    1.8
2018/02    1.9
2018/03    2.2
2018/04    2.3
2018/05    3.1
2018/06    2.7
2018/07    2.2
2018/08    2.4
Name: SO2, dtype: float64

### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [40]:
df.loc['2018/01':'2018/08',['SO2','CO']]

監測項目,SO2,CO
DATE,,
2018/01,1.8,0.34
2018/02,1.9,0.44
2018/03,2.2,0.40
2018/04,2.3,0.38
2018/05,3.1,0.34
2018/06,2.7,0.29
2018/07,2.2,0.21
2018/08,2.4,0.30
